In [3]:
import torch 
from torch import nn, optim
import pandas as pd
import numpy as np
import os

In [7]:
X_Kbest = pd.read_csv('../data/datasets/csv_files/Dataset_train.csv')

In [8]:
Y = pd.read_csv('../data/datasets/csv_files/test_feature_train.csv')

$$ \textbf{Préparation des données :} $$

In [14]:
feature_values = X_Kbest.values
label_values = Y.values

In [15]:
feature_tensor = torch.tensor(feature_values, dtype = torch.float32)
label_tensor = torch.tensor(label_values, dtype = torch.float32)

In [10]:
from torch.utils.data import Dataset, DataLoader

In [29]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [30]:
custom_dataset = CustomDataset(feature_tensor, label_tensor)

In [31]:
train_dataloader = DataLoader(dataset = custom_dataset, batch_size = 64, shuffle = True)

Choix de la sigmoide comme fonction d'activation car elle est utilisée pour les problèmes de classification binaire

In [23]:
input_dim = 10
hidden_dim = 10
output_dim = 1

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, hidden_dim)
        nn.init.kaiming_uniform_(self.layer_1.weight, nonlinearity="relu")
        #self.layer_2 = nn.Linear(hidden_dim, output_dim)
       
    def forward(self, x):
        #x = torch.nn.functional.relu(self.layer_1(x))
        x = torch.nn.functional.sigmoid(self.layer_1(x))

        return x

In [24]:
learning_rate = 0.1
model = NeuralNetwork(input_dim, hidden_dim, output_dim)
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [25]:
print(model)

NeuralNetwork(
  (layer_1): Linear(in_features=10, out_features=10, bias=True)
)


In [35]:
num_epochs = 100
loss_values = []


for epoch in range(num_epochs):
    model.train()
    for inputs,labels in train_dataloader:
        # zero the parameter gradients
        optimizer.zero_grad()
       
        # forward + backward + optimize
        pred = model(inputs)
        loss = loss_fn(pred, labels.unsqueeze(-1))
        #loss_values.append(loss.item())
        loss.backward()
        optimizer.step()

print("Training Complete")


ValueError: Using a target size (torch.Size([64, 1, 1])) that is different to the input size (torch.Size([64, 10])) is deprecated. Please ensure they have the same size.